# Azure Cognitive Search Vector Search Code Sample with Azure OpenAI
This code demonstrates how to use Azure Cognitive Search with OpenAI and Azure Python SDK
## Prerequisites
To run the code, install the following packages. Please note that the `pip install azure-search-documents==11.4.0a20230509004` is currently using the Dev Feed. For instructions on how to connect to the dev feed, please visit [Azure-Python-SDK Azure Search Documents Dev Feed](https://dev.azure.com/azure-sdk/public/_artifacts/feed/azure-sdk-for-python/connect/pip).

In [1]:
! pip install azure-search-documents==11.4.0a20230509004
! pip install openai
! pip install python-dotenv
! pip install tenacity

In [2]:
! pip install openai[datalib]

## Import required libraries and environment variables

In [2]:
# Import required libraries  
import os  
import json  
import openai  
from dotenv import load_dotenv  
from tenacity import retry, wait_random_exponential, stop_after_attempt  
from azure.core.credentials import AzureKeyCredential  
from azure.search.documents import SearchClient  
from azure.search.documents.indexes import SearchIndexClient  
from azure.search.documents.models import Vector  
from azure.search.documents.indexes.models import (  
    SearchIndex,  
    SearchField,  
    SearchFieldDataType,  
    SimpleField,  
    SearchableField,  
    SearchIndex,  
    SemanticConfiguration,  
    PrioritizedFields,  
    SemanticField,  
    SearchField,  
    SemanticSettings,  
    VectorSearch,  
    VectorSearchAlgorithmConfiguration,  
)  
  
# Configure environment variables  
load_dotenv()  
service_endpoint = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")  
index_name = os.getenv("AZURE_SEARCH_INDEX_NAME")  
key = os.getenv("AZURE_SEARCH_ADMIN_KEY") 
AZURE_OPENAI_CHATGPT_DEPLOYMENT = os.environ.get("AZURE_OPENAI_CHATGPT_DEPLOYMENT") or "gpt35-turbo-test" 
openai.api_type = "azure"  
openai.api_key = os.getenv("OPENAI_API_KEY")  
openai.api_base = os.getenv("OPENAI_ENDPOINT")  
openai.api_version = os.getenv("OPENAI_API_VERSION")  
#print(f"AZURE_SEARCH_ADMIN_KEY: [{key}]") 
credential = AzureKeyCredential(key)

In [8]:
# ChatGPT uses a particular set of tokens to indicate turns in conversations
prompt_prefix = """<|im_start|>system
Assistant helps the company employees with their questions on company policies, roles. 
Answer ONLY with the facts listed in the list of sources below. If there isn't enough information below, say you don't know. Do not generate answers that don't use the sources below. If asking a clarifying question to the user would help, ask the question. 
Each source file has a name followed by colon and by source page  followed by second colon and the actual information, always include the source name for each fact you use in the response. Use square brakets to reference the source file and source page, separate souefile and sourcepage by colon, e.g. [role_library.pdf:role_library-6.pdf]. Don't combine sources, list each source separately, e.g. [role_library.pdf:role_library-1.pdf][role_library.pdf:role_library-6.pdf].

Sources:
{sources}

<|im_end|>"""

turn_prefix = """
<|im_start|>user
"""

turn_suffix = """
<|im_end|>
<|im_start|>assistant
"""

prompt_history = turn_prefix

history = []

summary_prompt_template = """Below is a summary of the conversation so far, and a new question asked by the user that needs to be answered by searching in a knowledge base. Generate a search query based on the conversation and the new question. Source names are not good search terms to include in the search query.

Summary:
{summary}

Question:
{question}

Search query:
"""

## Create your search index
Create your search index schema and vector search configuration:

In [5]:
# Create a search index
index_client = SearchIndexClient(
    endpoint=service_endpoint, credential=credential)
fields = [
    SimpleField(name="id", type=SearchFieldDataType.String, key=True),
#    SearchableField(name="title", type=SearchFieldDataType.String,
#                    searchable=True, retrievable=True),
    SearchableField(name="content", type=SearchFieldDataType.String,
                    searchable=True, retrievable=True),
    SearchableField(name="category", type=SearchFieldDataType.String,
                    filterable=True, searchable=True, retrievable=True),
    SimpleField(name="sourcepage", type="Edm.String", filterable=True, facetable=True),
    SimpleField(name="sourcefile", type="Edm.String", filterable=True, facetable=True),
#    SearchField(name="titleVector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
#                searchable=True, dimensions=1536, vector_search_configuration="my-vector-config"),
    SearchField(name="contentVector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                searchable=True, dimensions=1536, vector_search_configuration="my-vector-config"),
]

vector_search = VectorSearch(
    algorithm_configurations=[
        VectorSearchAlgorithmConfiguration(
            name="my-vector-config",
            kind="hnsw",
            hnsw_parameters={
                "m": 4,
                "efConstruction": 400,
                "efSearch": 1000,
                "metric": "cosine"
            }
        )
    ]
)

semantic_config = SemanticConfiguration(
    name="my-semantic-config",
    prioritized_fields=PrioritizedFields(
#        title_field=SemanticField(field_name="title"),
        prioritized_keywords_fields=[SemanticField(field_name="category")],
        prioritized_content_fields=[SemanticField(field_name="content")]
    )
)

# Create the semantic settings with the configuration
semantic_settings = SemanticSettings(configurations=[semantic_config])

# Create the search index with the semantic settings
index = SearchIndex(name=index_name, fields=fields,
                    vector_search=vector_search, semantic_settings=semantic_settings)
result = index_client.create_or_update_index(index)
print(f' {result.name} created')


 test-vector-index-pdf created


## Perform a vector similarity search

In [9]:
# Pure Vector Search
#user_input = "What is the difference in responsibilities between a Vice President of Human Resources and Manager of Human Resources"  
user_input = "What are responsibilities of a Vice President of Human Resources and Manager of Human Resources"  

# Exclude category, to simulate scenarios where there's a set of docs you can't see
exclude_category = None

query = user_input

search_client = SearchClient(service_endpoint, index_name, AzureKeyCredential(key))  

def generate_embeddings(text):
    response = openai.Embedding.create(
        input=text, engine="ada-embed-test")
    embeddings = response['data'][0]['embedding']
    return embeddings

r = search_client.search(  
    search_text="",  
    vector=Vector(value=generate_embeddings(query), k=6, fields="contentVector"),  
    select=["content", "sourcefile", "sourcepage"] 
)  

results = [doc['sourcefile'] + ": "+doc['sourcepage'] + ": " + doc['content'].replace("\n", "").replace("\r", "") for doc in r]

content = "\n".join(results)

prompt = prompt_prefix.format(sources=content) + prompt_history + user_input + turn_suffix

completion = openai.Completion.create(
    engine=AZURE_OPENAI_CHATGPT_DEPLOYMENT, 
    prompt=prompt, 
    temperature=0.7, 
    max_tokens=1024,
    stop=["<|im_end|>", "<|im_start|>"])

prompt_history += user_input + turn_suffix + completion.choices[0].text + "\n<|im_end|>" + turn_prefix
history.append("user: " + user_input)
history.append("assistant: " + completion.choices[0].text)

print("\n-------------------\n".join(history))
print("\n-------------------\nSource:\n" + prompt)
print("\n-------------------\nPrompt:\n" + prompt)

"""for result in results:  
    print(f"sourcepage: {result['sourcepage']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Content: {result['content']}")  
    print(f"Category: {result['category']}\n")  """


user: What are responsibilities of a Vice President of Human Resources and Manager of Human Resources
-------------------
assistant: The responsibilities of a Vice President of Human Resources [role_library.pdf: role_library-8.pdf] include:
- Developing, implementing and monitoring comprehensive HR strategies and initiatives
- Collaborating with other departments to ensure alignment of HR initiatives with the company’s overall strategy
- Overseeing the recruitment and onboarding process
- Developing, implementing and monitoring training and development initiatives
- Managing employee relations, including conflict resolution, disciplinary action and performance management
- Developing and implementing compensation and benefit plans
- Tracking and analyzing HR metrics
- Ensuring compliance with all applicable laws and regulations
- Developing and monitoring HR budgets
- Staying up-to-date on the latest HR trends and best practices

The responsibilities of a Manager of Human Resources [ro

'for result in results:  \n    print(f"sourcepage: {result[\'sourcepage\']}")  \n    print(f"Score: {result[\'@search.score\']}")  \n    print(f"Content: {result[\'content\']}")  \n    print(f"Category: {result[\'category\']}\n")  '

## Perform a vector similarity search

In [6]:
# Pure Vector Search multi-lingual (e.g 'tools for software development' in Dutch)  
query = "tools voor softwareontwikkeling"  
  
search_client = SearchClient(service_endpoint, index_name, AzureKeyCredential(key))  
  
results = search_client.search(  
    search_text="",  
    vector=Vector(value=generate_embeddings(query), k=3, fields="contentVector"),  
    select=["title", "content", "category"]  
)  
  
for result in results:  
    print(f"Title: {result['title']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Content: {result['content']}")  
    print(f"Category: {result['category']}\n")  


Title: Azure DevOps
Score: 0.80402035
Content: Azure DevOps is a suite of services that help you plan, build, and deploy applications. It includes Azure Boards for work item tracking, Azure Repos for source code management, Azure Pipelines for continuous integration and continuous deployment, Azure Test Plans for manual and automated testing, and Azure Artifacts for package management. DevOps supports a wide range of programming languages, frameworks, and platforms, making it easy to integrate with your existing development tools and processes. It also integrates with other Azure services, such as Azure App Service and Azure Functions.
Category: Developer Tools

Title: Azure DevTest Labs
Score: 0.79679435
Content: Azure DevTest Labs is a fully managed service that enables you to create, manage, and share development and test environments in Azure. It provides features like custom templates, cost management, and integration with Azure DevOps. DevTest Labs supports various platforms, suc

## Perform a Pure Vector Search with a filter

In [7]:
# Pure Vector Search with Filter
query = "tools for software development"  
  
search_client = SearchClient(service_endpoint, index_name, AzureKeyCredential(key))  
  
results = search_client.search(  
    search_text="",  
    vector=Vector(value=generate_embeddings(query), k=3, fields="contentVector"),  
    filter="category eq 'Developer Tools'",
    select=["title", "content", "category"] 
)  
  
for result in results:  
    print(f"Title: {result['title']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Content: {result['content']}")  
    print(f"Category: {result['category']}\n")  


Title: Azure DevOps
Score: 0.82971567
Content: Azure DevOps is a suite of services that help you plan, build, and deploy applications. It includes Azure Boards for work item tracking, Azure Repos for source code management, Azure Pipelines for continuous integration and continuous deployment, Azure Test Plans for manual and automated testing, and Azure Artifacts for package management. DevOps supports a wide range of programming languages, frameworks, and platforms, making it easy to integrate with your existing development tools and processes. It also integrates with other Azure services, such as Azure App Service and Azure Functions.
Category: Developer Tools

Title: Azure DevTest Labs
Score: 0.81866795
Content: Azure DevTest Labs is a fully managed service that enables you to create, manage, and share development and test environments in Azure. It provides features like custom templates, cost management, and integration with Azure DevOps. DevTest Labs supports various platforms, suc

## Perform a Hybrid Search

In [8]:
# Hybrid Search
query = "scalable storage solution"  
  
search_client = SearchClient(service_endpoint, index_name, AzureKeyCredential(key))  
  
results = search_client.search(  
    search_text=query,  
    vector=Vector(value=generate_embeddings(query), k=3, fields="contentVector"),  
    select=["title", "content", "category"],
    top=3
)  
  
for result in results:  
    print(f"Title: {result['title']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Content: {result['content']}")  
    print(f"Category: {result['category']}\n")  


Title: Azure Storage
Score: 0.03306011110544205
Content: Azure Storage is a scalable, durable, and highly available cloud storage service that supports a variety of data types, including blobs, files, queues, and tables. It provides a massively scalable object store for unstructured data. Storage supports data redundancy and geo-replication, ensuring high durability and availability. It offers a variety of data access and management options, including REST APIs, SDKs, and Azure Portal. You can secure your data using encryption at rest and in transit.
Category: Storage

Title: Azure Table Storage
Score: 0.032258063554763794
Content: Azure Table Storage is a fully managed, NoSQL datastore that enables you to store and query large amounts of structured, non-relational data. It provides features like automatic scaling, schema-less design, and a RESTful API. Table Storage supports various data types, such as strings, numbers, and booleans. You can use Azure Table Storage to store and manage

## Perform a Semantic Hybrid Search

In [13]:
# Semantic Hybrid Search
query = "what is azure sarch?"

search_client = SearchClient(
    service_endpoint, index_name, AzureKeyCredential(key))

results = search_client.search(
    search_text=query,
    vector=Vector(value=generate_embeddings(
        query), k=3, fields="contentVector"),
    select=["title", "content", "category"],
    query_type="semantic", query_language="en-us", semantic_configuration_name='my-semantic-config', query_caption="extractive", query_answer="extractive",
    top=3
)

semantic_answers = results.get_answers()
for answer in semantic_answers:
    if answer.highlights:
        print(f"Semantic Answer: {answer.highlights}")
    else:
        print(f"Semantic Answer: {answer.text}")
    print(f"Semantic Answer Score: {answer.score}\n")

for result in results:
    print(f"Title: {result['title']}")
    print(f"Content: {result['content']}")
    print(f"Category: {result['category']}")

    captions = result["@search.captions"]
    if captions:
        caption = captions[0]
        if caption.highlights:
            print(f"Caption: {caption.highlights}\n")
        else:
            print(f"Caption: {caption.text}\n")


Semantic Answer: Azure Cognitive Search is<em> a fully managed search-as-a-service that enables you to build rich search experiences for your applications.</em> It provides features like full-text search, faceted navigation, and filters. Azure Cognitive Search supports various data sources, such as Azure SQL Database, Azure Blob Storage, and Azure Cosmos DB.
Semantic Answer Score: 0.9462890625

Title: Azure Cognitive Search
Content: Azure Cognitive Search is a fully managed search-as-a-service that enables you to build rich search experiences for your applications. It provides features like full-text search, faceted navigation, and filters. Azure Cognitive Search supports various data sources, such as Azure SQL Database, Azure Blob Storage, and Azure Cosmos DB. You can use Azure Cognitive Search to index your data, create custom scoring profiles, and integrate with other Azure services. It also integrates with other Azure services, such as Azure Cognitive Services and Azure Machine Lea